# Toto Expenses: Category Predictor

In [249]:
import sys
import pickle

In [250]:
import json
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings

nltk.download('stopwords')
nltk.download('punkt')

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


---
## Data Analysis

In [251]:
# Ingestion of the Expenses in dict objects
def transform_line(line): 
    expense = json.loads(line)
    return expense

In [252]:
# Read the data from the backup file
# Save it into a list
def ingest_data(filename):
    data = []
    
    with open(filename, "r") as file: 
        for idx, line in enumerate(file):
            expense = transform_line(line)
            
            data.append(expense)
    
    return data
        

In [253]:
def to_dataframe(data: list):
    return pd.DataFrame(data)
    

In [254]:
df = to_dataframe(ingest_data("expenses.json"))

In [255]:
df.head(2)

,_id,amount,date,category,description,creditMom,creditOther,yearMonth,currency,amountInEuro,user,monthly,subscriptionId,consolidated,cardId,cardMonth,weekendId,cardYear,additionalData,tags
0,583dd4f72ab79c00017d9895,14.35,20161128,SUPERMERCATO,Supermercato (Esselunga),NaN,NaN,201612,EUR,14.35,nicolas.matteazzi@gmail.com,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,583dd5382ab79c00017d9897,15.00,20161128,VIAGGI,Ricarica Prepagata (Boston hotel),NaN,NaN,201612,EUR,15.00,nicolas.matteazzi@gmail.com,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
features = df[df["user"] == "nicolas.matteazzi@gmail.com"][["description", "category"]]
features.tail(5)

,description,category
5776,Google Photos,SVAGO
5777,Adobe,SVAGO
5778,Car insurance,AUTO
5779,Apple Tv+,SVAGO
5780,Mc Donald,FOOD


### Vectorizing the Descriptions
The vectorization process basically:
 * splits all descriptions into tokens and 
 * perform **stemming**


In [259]:
stopwords_vocab = set(stopwords.words("english")).union(set(stopwords.words("italian")))
# stopwords_vocab = set(stopwords.words("danish"))
punctuation_symbols = list(string.punctuation)

In [260]:
stemmer = PorterStemmer()

In [261]:
def tokenize(desc): 
    '''
    Tokenizes a description and applies the following:
     - removes stop words
     - removes useless characters (e.g. '-')
     - stems the word
    '''
    # Split descriptions into tokens
    tokens = desc.split()
    
    # Prepare to remove any punctuation in the word
    translation_table = str.maketrans('', '', ''.join(punctuation_symbols))
    
    # Filter out stopwords, remove numeric characters from the word
    tokens = [re.sub(r'\d', '', word.lower().translate(translation_table)) for word in tokens if word.lower() not in stopwords_vocab]
    tokens = [word for word in tokens if word not in punctuation_symbols and not word.isdigit()]
    
    
    # Perform stemming
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Join the tokens, as needed by TF-IDF
    tokens = " ".join(tokens)
    
    return tokens

In [262]:
# Create a new column "tokens" that contains the processed descriptions
features.loc[:,"tokens"] = features["description"].apply(tokenize)

In [271]:
range_low = np.random.randint(0, len(features))
features.iloc[range_low:range_low + 10]

,description,category,tokens
39,Sciarpa Caro Zara,XMAS,sciarpa caro zara
40,Yamamay intimo Caro,XMAS,yamamay intimo caro
41,Parcheggio Lampugnano,AUTO,parcheggio lampugnano
42,Pedaggi Autostrada Novembre,AUTO,pedaggi autostrada novembr
43,Parcheggio Lampugnano,AUTO,parcheggio lampugnano
44,Prelievo Bancomat (Sci),SVAGO,prelievo bancomat sci
45,Pagamento Elettricità,HOME,pagamento elettricità
46,Gas,HOME,ga
47,Mc Donald's,FOOD,mc donald
48,Benzina per Cervinia,SVAGO,benzina cervinia


In [272]:
# Split the dataset between test and train
train_df, test_df = train_test_split(features, test_size=0.2)

In [274]:
# Vectorize the tokens both for test and train datasets
tfidf_vectorizer = TfidfVectorizer()

X_train = tfidf_vectorizer.fit_transform(train_df["tokens"])

# Here we only transform because we need to consider the test set as if it was 
# a new set of data, not seen by the model
X_test = tfidf_vectorizer.transform(test_df["tokens"])


In [277]:
model = MultinomialNB()
model.fit(X_train, train_df["category"])

MultinomialNB()

In [278]:
# Evaluate the accuracy of the model
# 1. Predict 
Y_test = model.predict(X_test)

# 2. Evaluate
accuracy = accuracy_score(test_df["category"], Y_test)

print(accuracy)

0.7193995381062356


### Trying a different vectorizer

In [279]:
binary_vectorizer = CountVectorizer(binary=True)

X_train = binary_vectorizer.fit_transform(train_df["tokens"])
X_test = binary_vectorizer.transform(test_df["tokens"])


In [285]:
train_df["tokens"]

4592              medicin oper
760                       park
1081                   contabo
116      parcheggio lampugnano
5697      hk handel og kontorf
                 ...          
4406               super vacat
421          boat tour navigli
2011    black star burger perm
4648               supermarket
1623              pant bertoni
Name: tokens, Length: 3464, dtype: object

In [280]:
model = MultinomialNB()
model.fit(X_train, train_df["category"])

MultinomialNB()

In [281]:
# Evaluate the accuracy of the model
# 1. Predict 
Y_test = model.predict(X_test)
Y_train = model.predict(X_train)

# 2. Evaluate
accuracy = accuracy_score(test_df["category"], Y_test)
train_accuracy = accuracy_score(train_df["category"], Y_train)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Training accuracy: 0.8585
Test accuracy: 0.7425


In [282]:
comparison_df = pd.DataFrame(data={"Tokens": test_df["tokens"], "Expected Category": test_df["category"], "Predicted Category": Y_test})

In [283]:
comparison_df[comparison_df["Expected Category"] != comparison_df["Predicted Category"]].head(50)

,Tokens,Expected Category,Predicted Category
5535,sport outlet ishoej ishoej,VARIE,AUTO
4109,ejendomskat,HOME,VARIE
1168,cafè domenica,SVAGO,FOOD
5174,,VARIE,AUTO
1429,silvan,FURNITURE,HOME
3893,insur etc,HOME,AUTO
3660,dog food,PET,VARIE
296,cena carolin mexicali,USCITE,VARIE
5267,amzn mktp de uf,VARIE,SVAGO
898,parch mxp,AUTO,VIAGGI


### Trying One-hot encoding

In [298]:
oh_train_df = train_df.copy()

oh_train_df["split_tokens"] = oh_train_df["tokens"].apply(str.split)
oh_train_df["split_tokens"]

4592                [medicin, oper]
760                          [park]
1081                      [contabo]
116        [parcheggio, lampugnano]
5697      [hk, handel, og, kontorf]
                   ...             
4406                 [super, vacat]
421           [boat, tour, navigli]
2011    [black, star, burger, perm]
4648                  [supermarket]
1623                [pant, bertoni]
Name: split_tokens, Length: 3464, dtype: object

In [312]:
def create_vocab(list_of_tokens): 
    """Creates a vocabulary dict of the list of tokens used in the training data. 
    
    The dictionnary is an index dictionnary, that assign to each word an index. 
    The dictionnary is meant to be used for "one-hot encoding" or similar
    
    Returns
     - (dict) where the key is the word and the value is the index
    """
    
    vocab = {}
    
    idx = 1
    for l in list_of_tokens:
        for token in l: 
            if token not in vocab.keys(): 
                vocab[token] = idx
                idx += 1
                
    # Add the UNK word
    vocab["UNK"] = 0
    
    return vocab

In [313]:
vocab = create_vocab(oh_train_df["split_tokens"].values)

In [336]:
def custom_encode(words: list, vocab: dict, unknown_word: str = "UNK"): 
    
    encoded_words = np.zeros(len(vocab))
    
    # Create an encoding of the words
    for word in words: 
        if word in vocab.keys():
            idx = vocab[word]
        else:
            idx = vocab[unknown_word]
        
        encoded_words[idx] = 1
    
    return encoded_words.tolist()
    
    

In [338]:
# Quick test
# custom_encode("parcheggio lampugnano".split(), vocab)

In [339]:
# Apply to training dataframe
oh_train_df["encoded_tokens"] = oh_train_df["split_tokens"].apply(lambda x: custom_encode(x, vocab))

In [342]:
X_train = oh_train_df["encoded_tokens"].values.tolist()
Y_train = oh_train_df["category"].values.tolist()

In [343]:
model = MultinomialNB()
model.fit(X_train, Y_train)

MultinomialNB()

In [344]:
# Evaluate the accuracy of the model
oh_test_df = test_df.copy()
oh_test_df["split_tokens"] = oh_test_df["tokens"].apply(str.split)
oh_test_df["encoded_tokens"] = oh_test_df["split_tokens"].apply(lambda x: custom_encode(x, vocab))

X_test = oh_test_df["encoded_tokens"].values.tolist()

# 1. Predict 
Y_test = model.predict(X_test)
Y_train_pred = model.predict(X_train)

# 2. Evaluate
accuracy = accuracy_score(oh_test_df["category"], Y_test)
train_accuracy = accuracy_score(oh_train_df["category"], Y_train)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Training accuracy: 1.0000
Test accuracy: 0.7471


# Saving the model

In [288]:
my_X = binary_vectorizer.transform(["benzina"])
model.predict(my_X)

array(['AUTO'], dtype='<U12')

In [90]:
with open("expcat-model-cbm", "wb") as f: 
    pickle.dump(model, f)

In [91]:
with open("expcat-vect-cbm", "wb") as  f:
    pickle.dump(binary_vectorizer, f)

In [94]:
stopwords_vocab = set(stopwords.words("danish"))

In [95]:
with open("expcat-stopwords-cbm", "wb") as f: 
    pickle.dump(stopwords_vocab, f)